# Imputation

Playground with some imputation strategies/algorithms before they go into the mundi library.

In [1]:
import pandas as pd
import numpy as np
import mundi
from collections import defaultdict
from mundi.plugins.epidemic.covid19 import brasil_io_clean_cases

In [2]:
try:
    data = pd.read_csv('~/caso_full.csv.gz')
except FileNotFoundError:
    data = pd.read_csv('https://data.brasil.io/dataset/covid19/caso_full.csv.gz')

In [3]:
cases = brasil_io_clean_cases(data)

In [4]:
col = "cases"
data = cases.pivot_table(
    index="id", columns="date", values=col, fill_value=-1
).sort_index()
# data = filling_policies.sum_children(data).reset_index()
# data = data.melt(id_vars=["id"], var_name="date", value_name=col)
# data = data[data[col] >= 0]
data

date,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,...,2021-04-27,2021-04-28,2021-04-29,2021-04-30,2021-05-01,2021-05-02,2021-05-03,2021-05-04,2021-05-05,2021-05-06
id,,,,,,,,,,,,,,,,,,,,,
BR-1100015,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,3353,3384,3406,3406,3429,3430,3435,3463,3476,3476
BR-1100023,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,18136,18169,18244,18244,18387,18387,18410,18536,18566,18566
BR-1100031,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,635,635,635,635,636,636,637,639,641,641
BR-1100049,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,11073,11160,11262,11262,11275,11275,11294,11229,11249,11249
BR-1100056,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,2039,2047,2061,2061,2069,2074,2077,2084,2088,2088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BR-5222005,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,580,594,606,606,606,606,606,606,606,606
BR-5222054,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1043,1048,1049,1056,1056,1057,1073,1072,1072,1072
BR-5222203,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,119,123,123,124,124,124,124,124,124,124


In [14]:
import sys; del sys.modules['mundi.impute']
from mundi.impute import impute_subregions

pd.concat([cases, impute_subregions(cases, 'BR', 'date', level=7)]).set_index(['id', 'date']).sort_index()

cases  deaths
id    date                      
BR    2020-02-25       1       0
      2020-02-26       1       0
      2020-02-27       1       0
      2020-02-28       2       0
      2020-02-29       2       0
...                  ...     ...
BR-TO 2021-05-02  160449    2559
      2021-05-03  160449    2559
      2021-05-04  160449    2559
      2021-05-05  160449    2559
      2021-05-06  160449    2559

[2300073 rows x 2 columns]

In [8]:
data = cases
parts = [cases]
for level in [7, 6, 5, 4, 3]:
    print('\n\n\nlevel', level)
    parents = mundi.regions_dataframe(['parent_id'], country_id='BR', level=level)['parent_id']
    print(parents)
    data = impute_parents(data, parents, by='date', agg='sum')
    parts.append(data)
    print(data)

pd.concat(parts).reset_index()[cases.columns]




level 7
id
BR-4108551    BR-410313
BR-2205532    BR-220311
BR-4123600    BR-410306
BR-2703502    BR-270309
BR-3514205    BR-350101
                ...    
BR-1706001    BR-170103
BR-3551900    BR-350105
BR-1716653    BR-170103
BR-4312955    BR-430109
BR-5214507    BR-520310
Name: parent_id, Length: 5570, dtype: object
             date         id   cases  deaths
0      2020-04-05  BR-110206       1       0
1      2020-04-06  BR-110206       1       0
2      2020-04-07  BR-110206       1       0
3      2020-04-08  BR-110206       1       0
4      2020-04-09  BR-110206       1       0
...           ...        ...     ...     ...
220542 2021-05-02  BR-530101  334625    7200
220543 2021-05-03  BR-530101  335325    7228
220544 2021-05-04  BR-530101  336008    7267
220545 2021-05-05  BR-530101  336008    7267
220546 2021-05-06  BR-530101  336008    7267

[220547 rows x 4 columns]



level 6
id
BR-130309    BR-1303
BR-250211    BR-2502
BR-240212    BR-2402
BR-170104    BR-1701
BR-430112   

,date,id,cases,deaths
0,2020-03-17,BR-1200401,3,0
1,2020-03-18,BR-1200401,3,0
2,2020-03-19,BR-1200401,4,0
3,2020-03-20,BR-1200401,7,0
4,2020-03-21,BR-1200401,11,0
...,...,...,...,...
2300068,2021-05-02,BR,14398814,395216
2300069,2021-05-03,BR,14431314,396178
2300070,2021-05-04,BR,14486350,398783
2300071,2021-05-05,BR,14538520,400757


In [97]:
mundi.regions_dataframe(['parent_id'], country_id='BR', level=4)['parent_id']

id
BR-AC           BR-1
BR-AL           BR-2
BR-AM           BR-1
BR-AP           BR-1
BR-BA           BR-2
               ...  
BR-SUS:3513    BR-SP
BR-SUS:3514    BR-SP
BR-SUS:3515    BR-SP
BR-SUS:3516    BR-SP
BR-SUS:3517    BR-SP
Name: parent_id, Length: 281, dtype: object

In [104]:
mundi.region('BR-2301').level, mundi.region('BR-CE').level

(4, 4)

In [32]:
df = cases.set_index(['id', 'date']).sort_index()

In [45]:
df.loc['BR-2208908'].reset_index().drop_duplicates('date').set_index('date')

,cases,deaths
date,,
2020-05-13,1,0
2020-05-14,1,0
2020-05-15,2,0
2020-05-16,2,0
2020-05-17,2,0
...,...,...
2021-05-02,619,8
2021-05-03,624,8
2021-05-04,624,8


In [106]:
cases.dtypes.to_dict()

{'date': dtype('<M8[ns]'),
 'id': StringDtype,
 'cases': dtype('int64'),
 'deaths': dtype('int64')}